In [25]:
%%markdown
source: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

NOTES:
* the ImageFolder class from torchvision.datasets works only when the data is in the current directory

source: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

NOTES:
* the ImageFolder class from torchvision.datasets works only when the data is in the current directory

In [22]:
# official version

from torchvision import transforms,datasets
import os

#data loader
data_transforms = {'train': transforms.Compose([transforms.RandomResizedCrop(224),transforms.RandomHorizontalFlip(),
                                           transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
                   'val' : transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),
                                                transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

data_dir = './hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
print(image_datasets)

<class 'torchvision.datasets.folder.ImageFolder'>
{'train': Dataset ImageFolder
    Number of datapoints: 244
    Root Location: ./hymenoptera_data/train
    Transforms (if any): Compose(
                             RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         )
    Target Transforms (if any): None, 'val': Dataset ImageFolder
    Number of datapoints: 153
    Root Location: ./hymenoptera_data/val
    Transforms (if any): Compose(
                             Resize(size=256, interpolation=PIL.Image.BILINEAR)
                             CenterCrop(size=(224, 224))
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
             

In [32]:
# test version
from torchvision import transforms,datasets
import os
import torch

#data loader

data_transform_train = transforms.Compose([transforms.RandomResizedCrop(224),transforms.RandomHorizontalFlip(),
                                           transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
data_transforms_test = transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),
                                                transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

data_dir_train = './hymenoptera_data/train'
image_datasets_train = datasets.ImageFolder(data_dir_train,data_transforms_train)

data_dir_test = './hymenoptera_data/val'
image_datasets_test = datasets.ImageFolder(data_dir_test,data_transforms_test)

data_loader_train = torch.utils.data.DataLoader(image_datasets_train,batch_size = 4,shuffle=True,num_workers=4)
data_loader_test = torch.utils.data.DataLoader(image_datasets_test,batch_size = 4,shuffle=True,num_workers=4)
